In [4]:
import torch 
from torch import nn, optim
import torch.nn.functional as F
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader

import numpy as np
import matplotlib.pyplot as plt

In [28]:
n_batches = 32
n_epochs = 5
lr = 1e-2

In [14]:

# Скачиваем размеченный датасет
img_transform = transforms.Compose([
    transforms.ToTensor()
])

dataset = MNIST('./data', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=n_batches, shuffle=True)

In [166]:
class Model(nn.Module):
    
    def __init__(self):
        super(Model, self).__init__()
        
        self.affine1_enc = nn.Linear(784, 100) 
        self.affine2_enc = nn.Linear(100, 16)
        
        self.affine1_dec = nn.Linear(16, 100)
        self.affine2_dec = nn.Linear(100, 784)
        
    def encoder(self, enc_inp):
        enc_out = F.elu( self.affine1_enc(enc_inp) )
        enc_out = torch.sigmoid( self.affine2_enc(enc_out) )
        
        return enc_out
    
    def decoder(self, dec_inp):
        
        dec_out = F.elu( self.affine1_dec(dec_inp) )
        dec_out = torch.sigmoid( self.affine2_dec(dec_out) )
        
        return dec_out
        
        
    def forward(self, x):
        # input shape: (32, 28, 28)
        out = self.encoder(x)
        # output shape: (32, 16)
#         torch.autograd.set_detect_anomaly(True)
        
        # add normal noise to it
#         zeros = torch.zeros(32, 16)
       
#         torch.autograd.set_detect_anomaly(True)
        
        
        out = self.decoder(out)
        
        return out
        
        
        
        
        

In [167]:
model = Model()

In [168]:
reconstrunction_loss = nn.MSELoss(reduction='sum')

In [169]:
opt = optim.Adam(model.parameters(), lr=lr)

In [180]:
losses = []

for ep in range(n_epochs):
    
    for batch_idx, data in enumerate(dataloader):
        img, _ = data
        
        img = img.view(img.size(0), -1)
        img = torch.autograd.Variable(img)
        
        opt.zero_grad()
        
        reconstruction = model.forward(img)
        
        assert reconstruction is not None, "reconstruction is nothing"
        
#         loss = nn.MSELoss(reconstruction, img)
        loss = reconstrunction_loss(reconstruction, img)
        
        loss.backward()
        
        opt.step()
        
        
    if ep % 2 == 0:
        print("ep #{} and loss is {}".format(ep, loss.item() / len(img)))
    

ep #0 and loss is 13.161482810974121


KeyboardInterrupt: 

In [ ]:
hashes = model.encoder(dataset.test_data)

hashes *= np.array([[2**bit for bit in range(16)]])

hashes = hashes.sum(axis=1)

In [173]:
torch.normal(mean=torch.arange(3., 3.), std=torch.arange(3., 3.))

tensor([])

In [175]:
len(dataloader)

1875